In [61]:
#import libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd 

In [62]:
# request access to website
url = 'https://www.cars.com/shopping/results/?page=70&page_size=20&list_price_max=&makes[]=&maximum_distance=20&models[]=&stock_type=all&zip='
response = requests.get(url)
if response.status_code == 200:
    print('webpage request successful')
else:
    print('webpage request failed')

webpage request successful


In [63]:
#list of required unique features
car_names = list ()
car_mileage = list()
car_rating = list()
car_rating_count = list()
car_price = list()
car_stock_type = list()
car_dealer_name = list()

In [25]:
# web page crawl loop for 70 pages
for page_no in range(1,70):
    # dynamic url for easy pagenation
    url = f'https://www.cars.com/shopping/results/?page={page_no}&page_size=20&list_price_max=&makes[]=&maximum_distance=20&models[]=&stock_type=all&zip='
    response = requests.get(url) #request site 
    soup = BeautifulSoup(response.content, 'html.parser') #soup object 

    car_cards = soup.findAll('div', {'class':'vehicle-card'})
    for car in car_cards:
        try:
            car_names.append(car.find('h2', {'class':'title'}).get_text())
        except:
            car_names.append('n/a')
        
        try:
            car_mileage.append(car.find('div', {'class':'mileage'}).get_text())
        except:
            car_mileage.append('n/a')
        
        try:
            car_rating.append(car.find('span', {'class':'sds-rating__count'}).get_text())
        except:
            car_rating.append('n/a')
        
        try:
            car_price.append(car.find('span', {'class':'primary-price'}).get_text())
        except:
            car_price.append('n/a')
        try:
            car_stock_type.append(car.find('p', {'class':'stock-type'}).get_text())
        except:
            car_stock_type.append('n/a')
        try:
            car_dealer_name.append(car.find('div', {'class':'dealer-name'}).get_text())
        except:
            car_dealer_name.append('n/a')
        try:
            car_rating_count.append(car.find('span', {'class':'sds-rating__link sds-button-link'}).get_text())
        except:
            car_rating_count.append('n/a')

In [40]:
#create data frame to structure data 
dataFrame = pd.DataFrame({'Name':car_names,'Dealer':car_dealer_name ,'Mileage':car_mileage, 'Rating':car_rating,'Rating Count':car_rating_count,'Price':car_price, 'Stock Type':car_stock_type})

In [64]:
#view data frame 
dataFrame.head()

,Name,Dealer,Mileage,Rating,Rating Count,Price,Stock Type
0,2017 Ford Escape Titanium,Monster Motors,73205,4.4,121,21683,Used
1,2018 Subaru Legacy Limited,EchoPark Automotive Centennial,30026,4.7,603,25899,Used
2,2020 Dodge Challenger SRT Hellcat,Rich Morton Mazda,10512,4.6,730,77272,Used
3,2019 Toyota Camry SE,Toyota of Plano,29444,4.8,"5,429",27051,Used
4,2017 Chevrolet Silverado 1500 LTZ,Wells Auto Group,52886,3.9,43,38999,Used


In [65]:
print(f'The data is made of {dataFrame.shape[0]} samples and {dataFrame.shape[1]} features') 

The data is made of 1994 samples and 7 features


In [55]:
#clean data 
dataFrame['Dealer'] = dataFrame['Dealer'].apply(lambda x: x.strip('\n').replace('\n',''))
dataFrame['Mileage'] = dataFrame['Mileage'].apply(lambda x: x.strip('mi.').replace(',',''))
dataFrame['Rating Count'] = dataFrame['Rating Count'].apply(lambda x: x.strip('reviews)').replace('(',''))
dataFrame['Price'] = dataFrame['Price'].apply(lambda x: x.strip('$').replace(',',''))

In [56]:
dataFrame

,Name,Dealer,Mileage,Rating,Rating Count,Price,Stock Type
0,2017 Ford Escape Titanium,Monster Motors,73205,4.4,121,21683,Used
1,2018 Subaru Legacy Limited,EchoPark Automotive Centennial,30026,4.7,603,25899,Used
2,2020 Dodge Challenger SRT Hellcat,Rich Morton Mazda,10512,4.6,730,77272,Used
3,2019 Toyota Camry SE,Toyota of Plano,29444,4.8,"5,429",27051,Used
4,2017 Chevrolet Silverado 1500 LTZ,Wells Auto Group,52886,3.9,43,38999,Used
...,...,...,...,...,...,...,...
1989,2016 Volvo XC90 T6 Momentum,M & I Motors,105390,4.5,390,26900,Used
1990,2016 Mercedes-Benz CLA-Class CLA 250 4MATIC,AutoNation Honda Renton,30888,4.6,785,27990,Used
1991,2019 Hyundai Tucson SE,Hyundai of 110,16958,4.5,53,22085,Used
1992,2018 BMW X1 xDrive28i,O'Meara Volkswagen of Thornton,31880,4.5,581,31441,Used


In [58]:
#save crawled data in directory
import os 
pwd = os.getcwd()
sav_dir = os.path.join(pwd,'crawled_data/cars.csv')
dataFrame.to_csv(sav_dir,index=False)